<a href="https://colab.research.google.com/github/KevinWang676/Retrieval-based-Voice-Conversion-New/blob/main/Sovits_for_Vast_AI_v7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 在终端里运行 apt install sudo 和 sudo apt-get install build-essential

In [ ]:
# 查看pytorch版本 建议2.0.1+cu118
import torch
print(torch.__version__)

In [ ]:
! nvidia-smi

In [ ]:
! python -m pip install -U pip setuptools wheel
! pip install -U so-vits-svc-fork
! pip install openai-whisper

In [ ]:
import subprocess
from pathlib import Path
import librosa
from scipy.io import wavfile
import numpy as np
import torch
import csv
import whisper



def split_long_audio(model, filepaths, character_name, save_dir="data_dir", out_sr=44100):
    if isinstance(filepaths, str):
        filepaths = [filepaths]

    for file_idx, filepath in enumerate(filepaths):

        save_path = Path(save_dir) / character_name
        save_path.mkdir(exist_ok=True, parents=True)

        print(f"Transcribing file {file_idx}: '{filepath}' to segments...")
        result = model.transcribe(filepath, word_timestamps=True, task="transcribe", beam_size=5, best_of=5)
        segments = result['segments']

        wav, sr = librosa.load(filepath, sr=None, offset=0, duration=None, mono=True)
        wav, _ = librosa.effects.trim(wav, top_db=20)
        peak = np.abs(wav).max()
        if peak > 1.0:
            wav = 0.98 * wav / peak
        wav2 = librosa.resample(wav, orig_sr=sr, target_sr=out_sr)
        wav2 /= max(wav2.max(), -wav2.min())

        for i, seg in enumerate(segments):
            start_time = seg['start']
            end_time = seg['end']
            wav_seg = wav2[int(start_time * out_sr):int(end_time * out_sr)]
            wav_seg_name = f"{character_name}_{file_idx}_{i}.wav"
            out_fpath = save_path / wav_seg_name
            wavfile.write(out_fpath, rate=out_sr, data=(wav_seg * np.iinfo(np.int16).max).astype(np.int16))


In [ ]:
"""
Here, we override config to have num_workers=0 because
of a limitation in HF Spaces Docker /dev/shm.
"""

import json
from pathlib import Path
import multiprocessing

def update_config(config_file="configs/44k/config.json"):
    config_path = Path(config_file)
    data = json.loads(config_path.read_text())
    data['train']['batch_size'] = 32
    data['train']['eval_interval'] = 500
    data['train']['num_workers'] = multiprocessing.cpu_count()
    data['train']['persistent_workers'] = True
#    data['train']['push_to_hub'] = True
#    data['train']['repo_id'] = MODEL_REPO_ID # tuple(data['spk'])[0]
#    data['train']['private'] = True
    config_path.write_text(json.dumps(data, indent=2, sort_keys=False))

In [ ]:
filenames = sorted(["syz0000.wav", "syz0001.wav", "syz0002.wav", "syz0003.wav", "syz0004.wav"]) # 只需上传wav文件；建议命名规则：syz_0000.wav, syz_0001.wav, syz_0002.wav, ...

In [ ]:
whisper_size = "medium"
whisper_model = whisper.load_model(whisper_size)

In [ ]:
split_long_audio(whisper_model, filenames, "syz", "dataset_raw") # 需要替换歌手名，原本是syz

In [ ]:
svc pre-resample
svc pre-config
svc pre-hubert -fm crepe

In [ ]:
update_config()

In [ ]:
svc train -t

In [ ]:
# train cluster

In [ ]:
! svc train-cluster

In [ ]:
from huggingface_hub import upload_file

upload_file(path_or_fileobj="/content/logs/44k/kmeans.pt", repo_id=MODEL_REPO_ID, path_in_repo="kmeans.pt")